# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,15.47,41,100,6.28,US,1701048430
1,1,adamstown,-25.0660,-130.1015,21.56,86,100,10.47,PN,1701048430
2,2,puerto natales,-51.7236,-72.4875,4.60,83,10,2.61,CL,1701048430
3,3,ushuaia,-54.8000,-68.3000,6.81,65,40,7.20,AR,1701048430
4,4,bilibino,68.0546,166.4372,-12.80,91,11,2.76,RU,1701048430


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    color='City',
    geo=True,
    tiles = "OSM",
    size='Humidity',
    frame_width=800,
    frame_height=500,
    xlabel='Longitude',
    ylabel='Latitude'
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,bandrele,-12.9067,45.1914,25.86,89,0,1.03,YT,1701048431
119,119,tolanaro,-25.0319,46.9987,24.04,88,0,3.09,MG,1701048438
132,132,brasilia,-15.7797,-47.9297,24.51,61,0,1.03,BR,1701047730
171,171,pasni,25.2631,63.4710,23.40,59,0,2.29,PK,1701048442
219,219,punta del este,-34.9667,-54.9500,21.06,82,0,3.60,UY,1701048446


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
18,bandrele,YT,-12.9067,45.1914,89,
119,tolanaro,MG,-25.0319,46.9987,88,
132,brasilia,BR,-15.7797,-47.9297,61,
171,pasni,PK,25.2631,63.4710,59,
219,punta del este,UY,-34.9667,-54.9500,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon}, {lat}, {radius}"
    params["bias"] = f"proximity:{lon}, {lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bandrele - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
brasilia - nearest hotel: No hotel found
pasni - nearest hotel: No hotel found
punta del este - nearest hotel: No hotel found
saint-joseph - nearest hotel: No hotel found
chiredzi - nearest hotel: No hotel found
ankazoabo - nearest hotel: No hotel found
gwanda - nearest hotel: No hotel found
sanya - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
sao jose do belmonte - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
bom jesus - nearest hotel: No hotel found
sao jose da coroa grande - nearest hotel: No hotel found
whakatane - nearest hotel: No hotel found
kimberley - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
18,bandrele,YT,-12.9067,45.1914,89,No hotel found
119,tolanaro,MG,-25.0319,46.9987,88,No hotel found
132,brasilia,BR,-15.7797,-47.9297,61,No hotel found
171,pasni,PK,25.2631,63.4710,59,No hotel found
219,punta del este,UY,-34.9667,-54.9500,82,No hotel found
250,saint-joseph,RE,-21.3667,55.6167,72,No hotel found
267,chiredzi,ZW,-21.0500,31.6667,73,No hotel found
278,ankazoabo,MG,-22.2833,44.5167,63,No hotel found
306,gwanda,ZW,-20.9333,29.0000,47,No hotel found
337,sanya,CN,18.2431,109.5050,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    color='City',
    geo=True,
    tiles = "OSM",
    size='Humidity',
    frame_width=800,
    frame_height=500,
    xlabel='Longitude',
    ylabel='Latitude',
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)